In [320]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/rahulprakash/Desktop/Datathon")  # Replace with your actual path

scoring = pd.read_csv("scoring.csv")
training = pd.read_csv("training.csv")

In [322]:
training = training[training['Fuel Type'] != 'Unknown']
training = training[training['Number of Vehicles Registered at the Same Address'] != 'Unknown']
training['Model Year Difference'] = training['Date'] - training['Model Year']

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population,Model Year Difference
0,2019,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,≥4,Statewide,395883,-1.0
1,2020,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,370954,0.0
2,2021,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,349406,1.0
3,2019,P,Not Applicable,Gasoline,2019.0,ICE,Not Applicable,≥4,Statewide,348475,0.0
4,2019,P,Not Applicable,Gasoline,2018.0,ICE,Not Applicable,≥4,Statewide,333296,1.0
...,...,...,...,...,...,...,...,...,...,...,...
41048,2019,B,Not Applicable,Diesel,1983.0,ICE,Not Applicable,1,Statewide,1,36.0
41049,2019,B,Not Applicable,Diesel,1980.0,ICE,Not Applicable,2,Statewide,1,39.0
41050,2019,B,Not Applicable,Diesel,1978.0,ICE,Not Applicable,3,Statewide,1,41.0
41051,2019,B,Not Applicable,Diesel,1976.0,ICE,Not Applicable,2,Statewide,1,43.0


In [324]:
#Impute GVWR Class - used to compute the relationship between categorical variables
import scipy.stats as stats

def cramers_v(chi2, n, dof):
    return np.sqrt(chi2 / (n * min(dof, 1)))

contingency_table = pd.crosstab(training['GVWR Class'], training['Number of Vehicles Registered at the Same Address'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
cramers_v_value = cramers_v(chi2, training.shape[0], dof)
cramers_v_value

0.05173856197673511

In [348]:
counts = training.groupby('GVWR Class')['Vehicle Category'].value_counts().unstack()
counts

Vehicle Category,B,BS,BT,MC,MH,P,T1,T2,T3,T4,T5,T6,T7
GVWR Class,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,3010.0,3492.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4071.0,3921.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1778.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1998.0,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2390.0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2302.0,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1854.0
Not Applicable,2306.0,2175.0,1920.0,1993.0,1922.0,3788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [349]:
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T1'), 'GVWR Class'] = '1'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T2'), 'GVWR Class'] = '1'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T3'), 'GVWR Class'] = '2'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T4'), 'GVWR Class'] = '2'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T5'), 'GVWR Class'] = '3'
training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T7'), 'GVWR Class'] = '8'

possible_values = ['4', '5', '6', '7']
probabilities = [0.25, 0.25, 0.25, 0.25] 
indices = training.loc[(training['GVWR Class'] == 'Unknown') & (training['Vehicle Category'] == 'T6')].index
training.loc[indices, 'GVWR Class'] = np.random.choice(possible_values, size=len(indices), p=probabilities)


In [350]:
counts = training.groupby('Fuel Technology')['Electric Mile Range'].value_counts().unstack()
counts

Electric Mile Range,0 to 50 miles,101 to 150 miles,51 to 100 miles,>150 miles,Not Applicable
Fuel Technology,,,,,
BEV,63.0,999.0,1113.0,1185.0,NaN
FCEV,NaN,NaN,4.0,121.0,NaN
ICE,NaN,NaN,NaN,NaN,36979.0
PHEV,414.0,NaN,56.0,NaN,NaN


In [352]:
training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Diesel') & (training['Fuel Technology'] == 'BEV'), 'Electric Mile Range'] = '51 to 100 miles'
training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Gasoline') & (training['Fuel Technology'] == 'BEV'), 'Electric Mile Range'] = '51 to 100 miles'
training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Hydrogen') & (training['Fuel Technology'] == 'FCEV'), 'Electric Mile Range'] = '>150 miles'



#Electric + BEV
possible_values = ["0 to 50 miles", "101 to 150 miles", "51 to 100 miles", ">150 miles"]
probabilities = [11/539, 162/539, 178/539, 188/539]
indices = training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Electric') & (training['Fuel Technology'] == 'BEV')].index
training.loc[indices, 'Electric Mile Range'] = np.random.choice(possible_values, size=len(indices), p=probabilities)

#Gas + PHEV
possible_values = ["0 to 50 miles", "51 to 100 miles"] 
probabilities = [364/412, 48/412] 
indices = training.loc[(training['Electric Mile Range'] == 'Unknown') & (training['Fuel Type'] == 'Gasoline') & (training['Fuel Technology'] == 'PHEV')].index
training.loc[indices, 'Electric Mile Range'] = np.random.choice(possible_values, size=len(indices), p=probabilities)


In [333]:
bins = [0, 50000, 100000, 150000, 200000, 250000, 300000, 350000, 400000, np.inf]  # Define range boundaries
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8]  # Assign a category number to each range

training['Population Category'] = pd.cut(training['Vehicle Population'], bins=bins, labels=labels, right=False)
training

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population,Model Year Difference,Population Category
0,2019,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,≥4,Statewide,395883,-1.0,7
1,2020,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,370954,0.0,7
2,2021,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,349406,1.0,6
3,2019,P,Not Applicable,Gasoline,2019.0,ICE,Not Applicable,≥4,Statewide,348475,0.0,6
4,2019,P,Not Applicable,Gasoline,2018.0,ICE,Not Applicable,≥4,Statewide,333296,1.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
41048,2019,B,Not Applicable,Diesel,1983.0,ICE,Not Applicable,1,Statewide,1,36.0,0
41049,2019,B,Not Applicable,Diesel,1980.0,ICE,Not Applicable,2,Statewide,1,39.0,0
41050,2019,B,Not Applicable,Diesel,1978.0,ICE,Not Applicable,3,Statewide,1,41.0,0
41051,2019,B,Not Applicable,Diesel,1976.0,ICE,Not Applicable,2,Statewide,1,43.0,0


In [353]:
median_model_year_diff = training.groupby('Vehicle Category')['Model Year Difference'].median()

training['Model Year Difference'] = training.apply(
    lambda row: median_model_year_diff[row['Vehicle Category']] if pd.isna(row['Model Year Difference']) else row['Model Year Difference'],
    axis=1
)

training['Model Year Difference'].isna().sum()

0

In [354]:
training.loc[training['Model Year'].isna(), 'Model Year'] = (
    training['Date'] - training['Model Year Difference']
)

In [355]:
counts = scoring.groupby('Model Year')['Fuel Type'].value_counts().unstack()
counts

Fuel Type,Diesel,Electric,Gasoline,Hydrogen,Natural Gas
Model Year,,,,,
1980.0,64.0,17.0,83.0,NaN,7.0
1981.0,41.0,5.0,68.0,NaN,NaN
1982.0,37.0,5.0,66.0,NaN,NaN
1983.0,41.0,4.0,65.0,NaN,NaN
1984.0,43.0,6.0,78.0,NaN,1.0
1985.0,47.0,6.0,69.0,NaN,NaN
1986.0,55.0,3.0,65.0,NaN,2.0
1987.0,61.0,6.0,70.0,NaN,2.0
1988.0,65.0,3.0,70.0,NaN,1.0


In [356]:
scoring.loc[(scoring['GVWR Class'] == 'Unknown') & (scoring['Vehicle Category'] == 'T1'), 'GVWR Class'] = '1'
scoring.loc[(scoring['GVWR Class'] == 'Unknown') & (scoring['Vehicle Category'] == 'T2'), 'GVWR Class'] = '1'
scoring.loc[(scoring['GVWR Class'] == 'Unknown') & (scoring['Vehicle Category'] == 'T3'), 'GVWR Class'] = '2'
scoring.loc[(scoring['GVWR Class'] == 'Unknown') & (scoring['Vehicle Category'] == 'T4'), 'GVWR Class'] = '2'
scoring.loc[(scoring['GVWR Class'] == 'Unknown') & (scoring['Vehicle Category'] == 'T5'), 'GVWR Class'] = '3'
scoring.loc[(scoring['GVWR Class'] == 'Unknown') & (scoring['Vehicle Category'] == 'T7'), 'GVWR Class'] = '8'

possible_values = ['4', '5', '6', '7'] 
probabilities = [0.25, 0.25, 0.25, 0.25]  
indices = scoring.loc[(scoring['GVWR Class'] == 'Unknown') & (scoring['Vehicle Category'] == 'T6')].index
scoring.loc[indices, 'GVWR Class'] = np.random.choice(possible_values, size=len(indices), p=probabilities)

In [357]:
scoring.loc[(scoring['Electric Mile Range'] == 'Unknown') & (scoring['Fuel Type'] == 'Diesel') & (scoring['Fuel Technology'] == 'BEV'), 'Electric Mile Range'] = '51 to 100 miles'
scoring.loc[(scoring['Electric Mile Range'] == 'Unknown') & (scoring['Fuel Type'] == 'Gasoline') & (scoring['Fuel Technology'] == 'BEV'), 'Electric Mile Range'] = '51 to 100 miles'
scoring.loc[(scoring['Electric Mile Range'] == 'Unknown') & (scoring['Fuel Type'] == 'Hydrogen') & (scoring['Fuel Technology'] == 'FCEV'), 'Electric Mile Range'] = '>150 miles'



#Electric + BEV
possible_values = ["0 to 50 miles", "101 to 150 miles", "51 to 100 miles", ">150 miles"] 
probabilities = [11/539, 162/539, 178/539, 188/539]  
indices = scoring.loc[(scoring['Electric Mile Range'] == 'Unknown') & (scoring['Fuel Type'] == 'Electric') & (scoring['Fuel Technology'] == 'BEV')].index
scoring.loc[indices, 'Electric Mile Range'] = np.random.choice(possible_values, size=len(indices), p=probabilities)

#Gas + PHEV
possible_values = ["0 to 50 miles", "51 to 100 miles"] 
probabilities = [364/412, 48/412]  
indices = scoring.loc[(scoring['Electric Mile Range'] == 'Unknown') & (scoring['Fuel Type'] == 'Gasoline') & (scoring['Fuel Technology'] == 'PHEV')].index
scoring.loc[indices, 'Electric Mile Range'] = np.random.choice(possible_values, size=len(indices), p=probabilities)

In [358]:
scoring['Model Year'] = scoring['Model Year'].apply(
    lambda x: np.random.randint(1980, 2027) if pd.isna(x) else x
)

nan_count = scoring['Model Year'].isna().sum()
nan_count

0

In [347]:
training.to_csv('training_cleaned.csv')
scoring.to_csv('scoring_cleaned.csv')
